In [1]:
import numpy as np
import matplotlib.pyplot as plt
import plumed
import pandas as pd
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
import math
import MDAnalysis as mda
from MDAnalysis.analysis import rms
from MDAnalysis.analysis.rms import RMSD
from MDAnalysis.analysis.rms import RMSF

In [2]:
PDB_CG = 'start_prot_CAL.pdb'
XTC_CG = 'trj_fit_BB_CAL.xtc'
u_CG = mda.Universe(str(PDB_CG), str(XTC_CG)) #we create the universe

PDB_AA = 'AA_sim/ref_nowat.pdb'
XTC_AA = 'AA_sim/trj_fit_BB.xtc'
u_AA = mda.Universe(str(PDB_AA), str(XTC_AA))

/Users/giuseppegambini/.local/lib/python3.8/site-packages/MDAnalysis/topology/PDBParser.py:328: UserWarning: Element information is missing, elements attribute will not be populated. If needed these can be guessed using MDAnalysis.topology.guessers.
  warnings.warn("Element information is missing, elements attribute "


In [ ]:
rmsf_cog = []

# Compute the CM of the reference configuration (frame zero)
cog_coordinates_ref = []
for i in range(len(u_AA.select_atoms("protein").residues)):
    residue_group = u_AA.select_atoms(f"resid {i+2} and name CA N C O HA HA1 HA2 HB HB1 HB2 HN")
    cog_coordinates_ref.append(residue_group.center_of_geometry())

In [ ]:
# Puoi fare delle prove con pochi residui per volta...

N_res = len(u_aa.select_atoms("protein").residues)

for i in range(N_res):
    residue_group = u_AA.select_atoms(f"resid {i+2} and name CA N C O HA HA1 HA2 HB HB1 HB2 HN")

    # define the array of positions 
    cog_coordinates = np.zeros((len(u_AA.trajectory), 3))

    # Loop for the RMSF of a single residue 
    for ts in u_AA.trajectory:
        # Compute center of mass of the selected residue 
        cog_coordinates[ts.frame] = residue_group.center_of_geometry()

    # Compute the RMSF for the single residue    
    rmsf_cog.append(np.mean(np.square(cog_coordinates - cog_coordinates_ref[i]), axis=0))
    print(f"RMSF of residue {i+2} computed")

In [ ]:
RMSF_aa = []

N_res = len(u_AA.select_atoms("protein").residues)

for i in range(N_res):
    RMSF_aa.append(math.sqrt(rmsf_cog[i][0] + rmsf_cog[i][1] + rmsf_cog[i][2]))

In [ ]:
# CG RMSF
BB = u_CG.select_atoms("name BB")
myRMSF_bb = RMSF(BB).run()

plt.rcParams["figure.figsize"] = (6, 4)
plt.plot(myRMSF_bb.rmsf, 'r', label="CG, backbone beads")
plt.plot(RMSF_aa, 'b', label="AA, backbone and hydrogens COG")
plt.xlabel('Residue index')
plt.ylabel(r'RMSF [$\AA$]')
plt.title('RMSF')
plt.legend(loc='best')
plt.grid()
#plt.savefig("pics/RMSF_AA_CG.png")